In [ ]:
def onebyone(mat_oco2,fh5):
    """
    Recibe la matriz de mediciones filtradas por processlist y el archivo h5 donde se va a escribir las mediciones
    Escribe las mediciones por latitud y longitud en el archivo h5
    """

    for ii,line in enumerate(mat_oco2):

        lat = line["/latitude"]
        lon = line["/longitude"]

        try:
            dset=fh5['%iN%iW' % (int(lat),int(lon))]
            n=dset.shape[0]
            dset.resize((n+1,))
            dset[n]=line
            fh5.flush()
        except:
            datalatlon=[line]
            try:
                dset=fh5.create_dataset('%iN%iW' % (int(lat),int(lon)),data=datalatlon,maxshape=(None,))

                fh5.flush()
            except:
                print 'nada works'


In [ ]:
def processlist(lista,h5filename):
    """
    Recibe una lista de archivos nc4 de oco2 para filtrarlos de acuerdo a latmin, latmax, lonmin y lonmax y el nombre del archivo h5 de salida
    Llama a onebyone
    """

    try:
        fh5=h5py.File(h5filename,'r+')
        print 'file exist'
    except:
        fh5=h5py.File(h5filename,'w')
        print 'new file'

    cont = 0
    for filename in lista:
        #print filename
        datos=h5py.File(filename,'r')
        lat = datos["/latitude"][()]
        lon = datos["/longitude"][()]
        xco2_qf = datos["/xco2_quality_flag"][()]
        cond_latlon = ((lat > latmin) & (lat < latmax) & (lon > lonmin) & (lon < lonmax) & (xco2_qf == 0))	### FILTRO PARA LATITUD, LONGITUD Y xco2_quality_flag = 0 (GOOD)
        #print filename,len(lat),len(lat[cond_latlon])
        if len(lat[cond_latlon]) > 0:          ### SOLO CUANDO AL APLICAR EL FILTRO SE OBTIENE AL MENOS UNA MEDICION
            mat_inter = np.empty(len(lat[cond_latlon]),dtype=oco2_co2_type)
            for name in oco2_co2_type.names[3:]:       ### NO CONSIDERA tepoch, lat y lon POR QUE NO ESTAN EN EL HDF DE oco2
                mat_inter[name] = datos[name][()][cond_latlon]
            mat_inter["tepoch"] = np.array([(dt.datetime(year=huh[0],month=huh[1],day=huh[2],hour=huh[3],minute=huh[4],second=huh[5],microsecond=huh[6]*1000) - to).total_seconds() for huh in mat_inter["/date"]])
            mat_inter["lat"] = mat_inter["/latitude"]
            mat_inter["lon"] = mat_inter["/longitude"]
            #print "mat_inter",mat_inter.shape
            if cont == 0:
                mat_oco2 = copy.copy(mat_inter)	### SE CREA LA MATRIZ DONDE SE CONCATENAN LOS DIFERENTES DIAS AL INICIO, CUANDO EL CONTADOR ES 0
            else:
                mat_oco2 = np.concatenate((mat_oco2,mat_inter),axis=0)	### SE CONCATENA LOS VALORES FILTRADOS DE CADA DIA DEL MES A LA MATRIZ mat_oco2
            cont = cont+len(lat[cond_latlon])
        else:
            continue

        datos.close()
    #print "Contador",cont
    print "mat_oco2",h5filename,mat_oco2.shape
    onebyone(mat_oco2,fh5)	### SE MANDA LLAMAR onebyone
    #zlevels=np.arange(20)
    #fh5.attrs.create('zlevels', zlevels, dtype=zlevels.dtype )
    fh5.close()	### CIERRA EL ARCHIVO h5 DONDE SE ESCRIBE



In [8]:
import numpy as np
import h5py

# test para leer, filtrar y escribir
inname = "f:\\jupyter\\test2.h5"
outname = "f:\\jupyter\\testout.h5"

latpoint = 19.483781                               # latitud del punto de interes
lonpoint = -99.147                                 # longitud del punto de interes
gridsize = 0.14

datos=h5py.File(inname,'r')

grid_max_lat = latpoint + gridsize
grid_min_lat = latpoint - gridsize
grid_max_lon = lonpoint - gridsize
grid_min_lon = lonpoint + gridsize

gridname = '%iN%iW' % (int(latpoint),int(lonpoint))
lat = datos[gridname]["/latitude"]
lon = datos[gridname]["/longitude"]



gridcond = ((lat <= grid_max_lat) & (lat >= grid_min_lat) & (lon <= grid_min_lon) & (lon >= grid_max_lon))

#filterdata = datos[gridname][gridcond]

print(datos[gridname][gridcond].shape, datos[gridname].shape)

fh5=h5py.File(outname,'w')
dset=fh5.create_dataset(gridname,data=datos[gridname][gridcond],maxshape=(None,))
fh5.flush()

datos.close()
fh5.close()

(42,) (114,)
